In [ ]:
import torch
from math import comb
import warnings
import os
import sys
warnings.filterwarnings('ignore')

project_root = os.path.abspath("..")  
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from src.cortical_estimator import CORTICAL
from src.utils import set_seed

In [ ]:
num_batches = 10
sample_size = 10

dimension = 1
num_thresholds = 1
alphabet_size = int(sum(comb(num_thresholds, k) for k in range(dimension + 1)))

if dimension == 2:
    box_param = 1.5 
elif dimension == 1:
    box_param = max(1.5,1.0 + int(num_thresholds/2) * 0.5)

params = sample_size, dimension, alphabet_size, num_thresholds, num_batches
cortical = CORTICAL(params=params,alphas=[1.0, 0.5, 0.1, 0.01], lambda_entropy=0.3,cost_coef=10.0,box_param=box_param)

In [ ]:
quant_points_test = [[-0.75],[0.75]]
thresholds_test = [[1.0,0.0]]
test_config_high = torch.cat([torch.tensor([20.0]),
                              torch.tensor(thresholds_test).view(num_thresholds*(dimension+1)),
                              torch.tensor(quant_points_test).view(alphabet_size*dimension)])
test_config_midL = torch.cat([torch.tensor([2.0]),
                              torch.tensor(thresholds_test).view(num_thresholds*(dimension+1)),
                              torch.tensor(quant_points_test).view(alphabet_size*dimension)])
test_config_midH = torch.cat([torch.tensor([7.0]),
                              torch.tensor(thresholds_test).view(num_thresholds*(dimension+1)),
                              torch.tensor(quant_points_test).view(alphabet_size*dimension)])
test_config_low = torch.cat([torch.tensor([-10.0]),
                             torch.tensor(thresholds_test).view(num_thresholds*(dimension+1)),
                              torch.tensor(quant_points_test).view(alphabet_size*dimension)])

In [ ]:
'''
verbose: 0 = no printing, 1 = printing epoch number and loss, 2 = printing all updates
test_params: None = no printing, test_config = printing MI, Px, and model used per epoch
'''
set_seed(42)

_,_,_ = cortical.train((-10.0,-2.0),epochs=100,subepochs=10,verbose=1,test_params=None)
_,_,_ = cortical.train((-2.0,4.0),epochs=100,subepochs=10,verbose=1,test_params=None)
_,_,_ = cortical.train((4.0,10.0),epochs=100,subepochs=10,verbose=1,test_params=None)
_,_,_ = cortical.train((10.0,20.0),epochs=100,subepochs=10,verbose=1,test_params=None)
cortical.save_models('./models/cortical_models')